## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Importing all dependancies

import pandas as pd
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, Dataset
import tqdm
import torch.optim as optim
from itertools import islice
import math

In [ ]:
## Accessing KG Node map dict
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_node_map_dict','rb')as f:
  KG_node_map_dict = pickle.load(f)

In [ ]:
## Accessing Trained Secondary Model

# Define the model architecture
class CFNet(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CFNet, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim, _weight=user_embeddings)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim, _weight=item_embeddings)
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embeddings(user_ids)
        item_embeds = self.item_embeddings(item_ids)
        x = torch.cat([user_embeds, item_embeds], dim=1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/secondary_model_tucson', 'rb') as handle:
    secondary_model = pickle.load(handle)

In [ ]:
## Accessing KG to secondary model map of users
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/user_kg_index_secondary_id_map_tucson','rb') as f:
  user_kg_index_secondary_id_map = pickle.load(f)

## Accessing KG to secondary model map of items
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/item_kg_index_secondary_id_map_tucson','rb') as f:
  item_kg_index_secondary_id_map = pickle.load(f)

## Definations

In [ ]:
## Function to get test POIs for a group
def get_test_poi_for_group(group):
  test_poi_group = []
  for user in group:
    for poi in userwise_test_poi[user]:
      test_poi_group.append(poi)

  return list(set(test_poi_group))

In [ ]:
def get_predicted_rating (user,poi):

  ## Script for poi rating prediction for a user from secondary model

  user_id = user_kg_index_secondary_id_map[KG_node_map_dict[user]]
  item_id = item_kg_index_secondary_id_map[KG_node_map_dict[poi]]

  user_tensor = torch.LongTensor([user_id])
  item_tensor = torch.LongTensor([item_id])

  user_embed = secondary_model.user_embeddings(user_tensor)
  item_embed = secondary_model.item_embeddings(item_tensor)

  x = torch.cat([user_embed, item_embed], dim=1)

  x = nn.functional.relu(secondary_model.fc1(x))
  x = secondary_model.fc2(x)

  #rating_pred = float(x)
  user_tensor.detach()
  item_tensor.detach()
  user_embed.detach()
  item_embed.detach()
  x.detach()
  return x

In [ ]:
## Function to get top K POI recommendation for group
def get_grp_recommendations(group_idx, candidate_grp_pois,K):
    #group_embedding = group_embeddings[group_idx]
    group_embedding = H1_grp_embedding
    predicted_ratings = []
    for poi in candidate_grp_pois:
        if poi in KG_node_map_dict and KG_node_map_dict[poi] in item_kg_index_secondary_id_map:
            item_id = item_kg_index_secondary_id_map[KG_node_map_dict[poi]]
            item_tensor = torch.LongTensor([item_id])
            item_embed = secondary_model.item_embeddings(item_tensor)[0]
            x = torch.cat([group_embedding, item_embed], dim=0)
            x = nn.functional.relu(secondary_model.fc1(x))
            x = secondary_model.fc2(x)
            predicted_ratings.append(float(x))
    pairs = list(zip(predicted_ratings, candidate_grp_pois))
    sorted_pairs = sorted(pairs, reverse=True)
    top_items = [pair[1] for pair in sorted_pairs[:K]]
    return top_items

In [ ]:
# Hit_u = items in recommedation which are in test set of u/ total items in recommendation
def get_avg_hit_for_group(group_idx,recom_items):
  hits = []
  group = user_group[group_idx]
  for user in group:
    holdout_pois = userwise_test_poi[user]
    intersection = 0
    for poi in recom_items:
      if poi in holdout_pois:
        intersection+=1
    hits.append(intersection/len(holdout_pois))
  #print(hits)
  return sum(hits)/len(hits)

In [ ]:
def get_avg_ndcg_for_group(group_idx,recom_items,K):
  NDCGs = []
  group = user_group[group_idx]
  for user in group:
    IDCG_user = 0
    if K<= len(userwise_test_poi[user]):
      for i in range(K):
        IDCG_user+= 1/math.log2(i+2)
    else:
      for i in range(len(userwise_test_poi[user])):
        IDCG_user+= 1/math.log2(i+2)

    DCG_user = 0
    for i in range(len(recom_items)):
      recom_poi = recom_items[i]
      if recom_poi in userwise_test_poi[user]:
        DCG_user += 1/math.log2(i+2)
    NDCGs.append(DCG_user/IDCG_user)

  return sum(NDCGs)/len(NDCGs)

## Pipeline

In [ ]:
# Accessing POI metadata to create the list of unique POIs
import pandas as pd
POI_metadata = pd.read_csv(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_poi_metadata.csv')
poi_list = list(POI_metadata.iloc[:,0])
print("Total famous POIs :", len(poi_list))

Total famous POIs : 1359


In [ ]:
# Accessing heldout POIs for users
file = open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/UP_dict_test', "rb")
UP_dict_test = pickle.load(file)

## Creating userwise test POI dictonary
userwise_test_poi = {}
for user in UP_dict_test:
  userwise_test_poi[user] = []
  for record in UP_dict_test[user]:
    userwise_test_poi[user].append(record[0])

In [13]:
# Accessing user groups
file = open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_group_10', "rb")
user_group = pickle.load(file)

In [14]:
len(user_group)

127

In [ ]:
## Accessing Group Embeddings
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/group5_embeddings_tucson','rb')as f:
  group_embeddings = pickle.load(f)

## Inferencing

In [ ]:
temp_grp = user_group[40]
temp_grp

['1ZPqlcxGGwvvth08c7TZ_g',
 'q7mLzjNP01zHQ7pkbl4OVg',
 'aJYSt-c2QUC7J6z9ifqR2g',
 '_2vLAIVo5Ub5p_fYbRTr_w',
 'S_sfbYb5_BoSGS4rgZep3w']

In [ ]:
KCGRS_grp_embed = group_embeddings[40]
KCGRS_grp_embed

tensor([ 0.0749,  0.0711, -0.4057, -0.3237, -0.2450, -0.0669, -0.0339, -0.0849,
         0.0647, -0.3372,  0.1367,  0.2379,  0.3476, -0.1244,  0.2152, -0.0049,
        -0.0513,  0.4383,  0.0140, -0.3005,  0.1396, -0.0218,  0.0358, -0.0961,
        -0.3919,  0.2131,  0.2162, -0.0043,  0.3590,  0.4265,  0.3695, -0.1677,
         0.2125,  0.4445,  0.0336,  0.1198,  0.0838,  0.0796, -0.3672,  0.4107,
        -0.4129, -0.1850, -0.2511, -0.2989, -0.1996,  0.1707,  0.2679, -0.2600,
        -0.3353,  0.2897], requires_grad=True)

In [ ]:
candidate_pois = get_test_poi_for_group(user_group[40])
recommenadtions_10 = get_grp_recommendations(40,candidate_pois,10)
group_hit_10 = get_avg_hit_for_group(40,recommenadtions_10)
group_ndcg_10 = get_avg_ndcg_for_group(40,recommenadtions_10,10)
print("Hit", group_hit_10, " NDCG", group_ndcg_10)

Hit 0.475  NDCG 0.34811642872635973


In [ ]:
H1_grp_embedding = torch.tensor([-0.8627,  0.7181, -0.4258, -0.6483, -0.7583, -0.5441, -0.2898,  0.5487,
          0.8372, -0.5979, -0.7152,  0.8514,  0.4452, -0.0285,  0.5525, -0.7385,
         -0.4660,  0.7437,  0.6394, -0.6925,  0.2890,  0.5196,  0.7101, -0.6439,
         -0.7257,  0.7247,  0.6036,  0.6037, -0.7768,  0.4918,  0.2313,  0.6195,
          0.6982,  0.2106,  0.8956, -0.2667,  0.2889,  0.7538, -0.7771,  0.5843,
         -0.5827, -0.6182, -0.7626, -0.5698,  0.5567,  0.8276,  0.6555,  0.5338,
         -0.4492,  0.6458])

In [ ]:
candidate_pois = get_test_poi_for_group(user_group[40])
recommenadtions_10 = get_grp_recommendations(40,candidate_pois,10)
group_hit_10 = get_avg_hit_for_group(40,recommenadtions_10)
group_ndcg_10 = get_avg_ndcg_for_group(40,recommenadtions_10,10)
print("Hit", group_hit_10, " NDCG", group_ndcg_10)

Hit 0.475  NDCG 0.34382170641412174


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 5-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 896/896 [00:18<00:00, 48.09it/s]

----------------For Group Size 5-------------------------
Avg Group Hit K = 5 : 0.14378689481174844
Avg Group NDCG K = 5 : 0.22268006336367727


Avg Group Hit K = 10 : 0.28955008898949003
Avg Group NDCG K = 10 : 0.27220999598411527


Avg Group Hit K = 15 : 0.4230363121435889
Avg Group NDCG K = 15 : 0.32257596988107357


Avg Group Hit K = 20 : 0.5190548534918757
Avg Group NDCG K = 20 : 0.35894823603638154


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 6-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 693/693 [00:14<00:00, 46.24it/s]

----------------For Group Size 6-------------------------
Avg Group Hit K = 5 : 0.12067376946511378
Avg Group NDCG K = 5 : 0.18581732401200476


Avg Group Hit K = 10 : 0.24220078641583115
Avg Group NDCG K = 10 : 0.22694112198738797


Avg Group Hit K = 15 : 0.3631227593216501
Avg Group NDCG K = 15 : 0.2729773716794898


Avg Group Hit K = 20 : 0.4701880235466766
Avg Group NDCG K = 20 : 0.3137520098837977


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 7-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 508/508 [00:15<00:00, 32.41it/s]

----------------For Group Size 7-------------------------
Avg Group Hit K = 5 : 0.10483025822077154
Avg Group NDCG K = 5 : 0.16072244751453044


Avg Group Hit K = 10 : 0.2094994372500015
Avg Group NDCG K = 10 : 0.19637247162510238


Avg Group Hit K = 15 : 0.3130151991148956
Avg Group NDCG K = 15 : 0.2359748643517255


Avg Group Hit K = 20 : 0.41209044910123405
Avg Group NDCG K = 20 : 0.27355570723850653


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 8-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 352/352 [00:09<00:00, 36.28it/s]

----------------For Group Size 8-------------------------
Avg Group Hit K = 5 : 0.08928348914371338
Avg Group NDCG K = 5 : 0.14304391895531732


Avg Group Hit K = 10 : 0.17609911272157747
Avg Group NDCG K = 10 : 0.17129890197351513


Avg Group Hit K = 15 : 0.2655143588381081
Avg Group NDCG K = 15 : 0.20455729647984597


Avg Group Hit K = 20 : 0.35189156644176306
Avg Group NDCG K = 20 : 0.2369352472082304


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 9-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 235/235 [00:09<00:00, 25.12it/s]

----------------For Group Size 9-------------------------
Avg Group Hit K = 5 : 0.08157000164897907
Avg Group NDCG K = 5 : 0.12820280298893102


Avg Group Hit K = 10 : 0.16057834248869787
Avg Group NDCG K = 10 : 0.15432043203926774


Avg Group Hit K = 15 : 0.2381523817345414
Avg Group NDCG K = 15 : 0.18374734390452385


Avg Group Hit K = 20 : 0.318190937915259
Avg Group NDCG K = 20 : 0.21378919911172453


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 10-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 127/127 [00:04<00:00, 28.25it/s]

----------------For Group Size 10-------------------------
Avg Group Hit K = 5 : 0.07243584758461653
Avg Group NDCG K = 5 : 0.11700334474578286


Avg Group Hit K = 10 : 0.13994305544462066
Avg Group NDCG K = 10 : 0.13879888867984097


Avg Group Hit K = 15 : 0.21299630172913614
Avg Group NDCG K = 15 : 0.1654771933116088


Avg Group Hit K = 20 : 0.28307363945356045
Avg Group NDCG K = 20 : 0.19190141277653233


#### With Hetero Embeddings

In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 5 With Hetero Embeddings-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 896/896 [00:14<00:00, 59.81it/s]

----------------For Group Size 5 With Hetero Embeddings-------------------------
Avg Group Hit K = 5 : 0.14295220629960498
Avg Group NDCG K = 5 : 0.22214855288112414


Avg Group Hit K = 10 : 0.28700858883786756
Avg Group NDCG K = 10 : 0.27144159705045673


Avg Group Hit K = 15 : 0.41958432655575445
Avg Group NDCG K = 15 : 0.3217023670423798


Avg Group Hit K = 20 : 0.5171853237261768
Avg Group NDCG K = 20 : 0.3584351918471914


In [ ]:
groupwise_hit_5 = []
groupwise_ndcg_5 = []
groupwise_hit_10 = []
groupwise_ndcg_10 = []
groupwise_hit_15 = []
groupwise_ndcg_15 = []
groupwise_hit_20 = []
groupwise_ndcg_20 = []

for index in tqdm.tqdm(range(len(user_group))):
  candidate_pois = get_test_poi_for_group(user_group[index])
  recommenadtions_5 = get_grp_recommendations(index,candidate_pois,5)
  recommenadtions_10 = get_grp_recommendations(index,candidate_pois,10)
  recommenadtions_15 = get_grp_recommendations(index,candidate_pois,15)
  recommenadtions_20 = get_grp_recommendations(index,candidate_pois,20)

  group_hit_5 = get_avg_hit_for_group(index,recommenadtions_5)
  group_ndcg_5 = get_avg_ndcg_for_group(index,recommenadtions_5,5)
  group_hit_10 = get_avg_hit_for_group(index,recommenadtions_10)
  group_ndcg_10 = get_avg_ndcg_for_group(index,recommenadtions_10,10)
  group_hit_15 = get_avg_hit_for_group(index,recommenadtions_15)
  group_ndcg_15 = get_avg_ndcg_for_group(index,recommenadtions_15,15)
  group_hit_20 = get_avg_hit_for_group(index,recommenadtions_20)
  group_ndcg_20 = get_avg_ndcg_for_group(index,recommenadtions_20,20)

  groupwise_hit_5.append(group_hit_5)
  groupwise_ndcg_5.append(group_ndcg_5)
  groupwise_hit_10.append(group_hit_10)
  groupwise_ndcg_10.append(group_ndcg_10)
  groupwise_hit_15.append(group_hit_15)
  groupwise_ndcg_15.append(group_ndcg_15)
  groupwise_hit_20.append(group_hit_20)
  groupwise_ndcg_20.append(group_ndcg_20)

print("----------------For Group Size 10 With Hetero Embeddings-------------------------")
print('Avg Group Hit K = 5 :',sum(groupwise_hit_5)/len(groupwise_hit_5))
print('Avg Group NDCG K = 5 :',sum(groupwise_ndcg_5)/len(groupwise_ndcg_5))
print("\n")
print('Avg Group Hit K = 10 :',sum(groupwise_hit_10)/len(groupwise_hit_10))
print('Avg Group NDCG K = 10 :',sum(groupwise_ndcg_10)/len(groupwise_ndcg_10))
print("\n")
print('Avg Group Hit K = 15 :',sum(groupwise_hit_15)/len(groupwise_hit_15))
print('Avg Group NDCG K = 15 :',sum(groupwise_ndcg_15)/len(groupwise_ndcg_15))
print("\n")
print('Avg Group Hit K = 20 :',sum(groupwise_hit_20)/len(groupwise_hit_20))
print('Avg Group NDCG K = 20 :',sum(groupwise_ndcg_20)/len(groupwise_ndcg_20))

100%|██████████| 127/127 [00:04<00:00, 31.13it/s]


----------------For Group Size 10 With Hetero Embeddings-------------------------
Avg Group Hit K = 5 : 0.07298518816547356
Avg Group NDCG K = 5 : 0.1171350863286201


Avg Group Hit K = 10 : 0.1433332915369333
Avg Group NDCG K = 10 : 0.140624792801863


Avg Group Hit K = 15 : 0.21078602211202618
Avg Group NDCG K = 15 : 0.1660649438217241


Avg Group Hit K = 20 : 0.2794347209303201
Avg Group NDCG K = 20 : 0.19183214839350604
